In [1]:
import csv
import pandas as pd
import geopandas as gpd
import os

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/gf/by74yth12b961jz2g6yc7rs40000gn/T/ipykernel_35658/2636044192.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-

In [2]:
# distance function 
import math
def haversine(lat1, lon1, lat2, lon2):
# distance function of the two coordinates 
# lat1: latitude of the first coordinate
# lon1: longtitude of the first coordinate
# lat2:latitude of the second coordinate
# lon2: longtitude of the second coordinate
# return the straight distance of the two coordinate in (km)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    r = 6371.0

    distance = r * c

    return distance

def get_distance(entry):
    return entry[-1]

In [3]:
def find_nearest_location(locations):
#find the nearest stop
#location: a list of nearby stops within the search radius
#return the nearest stop
    nearest_location = None
    nearest_distance = float('inf')

    for location in locations:
        distance = location[-1]  # Get the distance from the last element
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_location = location

    return nearest_location

In [9]:
def merge_nearby_bus(input_file, transport_file, output_file):
#merge the nearest bus stop to the property dataset 
#input_file: path for the property file
#transport_file: path for the public transport file
#output_file: path for the merged file
    transport_data = gpd.read_file(transport_file)
    property_list = pd.read_csv(input_file)
    
    nearby_transport_coord=[]
    for count in range(len(property_list["Latitude"])):
        property_lat = float(property_list["Latitude"][count])
        property_long = float (property_list["longitude"][count])
        nearby_transport_name_coordinates=[]
        for count in range(len(transport_data['LONGITUDE'])):
            latitude_float = float(transport_data["LATITUDE"][count])
            longitude_float = float(transport_data["LONGITUDE"][count])
            name = transport_data["STOP_NAME"][count]
            distance = haversine(property_lat, property_long, latitude_float, longitude_float)
            if distance <= 3:
                nearby_transport_name_coordinates.append([name,latitude_float,longitude_float, distance])
        
        nearby_transport_coord.append(nearby_transport_name_coordinates)

    nearest=[]
    for i, nearby_transport_coord in enumerate(nearby_transport_coord):
        nearest_location = find_nearest_location(nearby_transport_coord)
        nearest.append(nearest_location)
        
    new_column_values = nearest  # Replace with your actual values
    property_list['Nearest_Bus_Stop'] = new_column_values

    if not os.path.exists("../data/curated/seventh_clean/added_bus"):
        os.makedirs("../data/curated/seventh_clean/added_bus")
    
    property_list.to_csv(output_file, index=False)


In [10]:
APAR_1_DIR = '../data/curated/sixth_clean/apartment_1.csv'
APAR_2_DIR = '../data/curated/sixth_clean/apartment_2.csv'
APAR_3_DIR = '../data/curated/sixth_clean/apartment_3.csv'
APAR_4_DIR = '../data/curated/sixth_clean/apartment_4.csv'
APAR_5_DIR = '../data/curated/sixth_clean/apartment_5.csv'

HOUSE_1_DIR = '../data/curated/sixth_clean/house_1.csv'
HOUSE_2_DIR = '../data/curated/sixth_clean/house_2.csv'
HOUSE_3_DIR = '../data/curated/sixth_clean/house_3.csv'
HOUSE_4_DIR = '../data/curated/sixth_clean/house_4.csv'

TOWN_HOUSE_DIR = '../data/curated/sixth_clean/town_house.csv'

BUS_DIR = '../data/landing/public_transport/bus/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_METRO_BUS_STOP.shp'

merge_nearby_bus(APAR_1_DIR, BUS_DIR, '../data/curated/seventh_clean/apartment_1.csv')
print('finish A1')
merge_nearby_bus(APAR_2_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/apartment_2.csv')
print('finish A2')
merge_nearby_bus(APAR_3_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/apartment_3.csv')
print('finish A3')
merge_nearby_bus(APAR_4_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/apartment_4.csv')
print('finish A4')
merge_nearby_bus(APAR_5_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/apartment_5.csv')
print('finish A5')
merge_nearby_bus(HOUSE_1_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/house_1.csv')
print('finish H1')
merge_nearby_bus(HOUSE_2_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/house_2.csv')
print('finish H2')
merge_nearby_bus(HOUSE_3_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/house_3.csv')
print('finish H3')
merge_nearby_bus(HOUSE_4_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/house_4.csv')
print('finish H4')
merge_nearby_bus(TOWN_HOUSE_DIR, BUS_DIR, '../data/curated/seventh_clean/added_bus/town_house.csv')
print('finish TH')

finish A1
finish A2
finish A3
finish A4
finish A5
finish H1
finish H2
finish H3
finish H4
finish TH


In [11]:
def merge_nearby_tram(input_file, transport_file, output_file):
#merge the nearest tram stop to the property dataset 
#input_file: path for the property file
#transport_file: path for the public transport file
#output_file: path for the merged file
    transport_data = gpd.read_file(transport_file)
    property_list = pd.read_csv(input_file)
    
    nearby_transport_coord=[]
    for count in range(len(property_list["Latitude"])):
        property_lat = float(property_list["Latitude"][count])
        property_long = float (property_list["longitude"][count])
        nearby_transport_name_coordinates=[]
        for count in range(len(transport_data['LONGITUDE'])):
            latitude_float = float(transport_data["LATITUDE"][count])
            longitude_float = float(transport_data["LONGITUDE"][count])
            name = transport_data["STOP_NAME"][count]
            distance = haversine(property_lat, property_long, latitude_float, longitude_float)
            if distance <= 5:
                nearby_transport_name_coordinates.append([name,latitude_float,longitude_float, distance])
        
        nearby_transport_coord.append(nearby_transport_name_coordinates)

    nearest=[]
    for i, nearby_transport_coord in enumerate(nearby_transport_coord):
        nearest_location = find_nearest_location(nearby_transport_coord)
        nearest.append(nearest_location)
        
    new_column_values = nearest  # Replace with your actual values
    property_list['Nearest_Tram_Stop'] = new_column_values


    if not os.path.exists("../data/curated/seventh_clean/added_bus_tram"):
        os.makedirs("../data/curated/seventh_clean/added_bus_tram")
    
    property_list.to_csv(output_file, index=False)

In [12]:
APAR_1_DIR = '../data/curated/seventh_clean/added_bus/apartment_1.csv'
APAR_2_DIR = '../data/curated/seventh_clean/added_bus/apartment_2.csv'
APAR_3_DIR = '../data/curated/seventh_clean/added_bus/apartment_3.csv'
APAR_4_DIR = '../data/curated/seventh_clean/added_bus/apartment_4.csv'
APAR_5_DIR = '../data/curated/seventh_clean/added_bus/apartment_5.csv'

HOUSE_1_DIR = '../data/curated/seventh_clean/added_bus/house_1.csv'
HOUSE_2_DIR = '../data/curated/seventh_clean/added_bus/house_2.csv'
HOUSE_3_DIR = '../data/curated/seventh_clean/added_bus/house_3.csv'
HOUSE_4_DIR = '../data/curated/seventh_clean/added_bus/house_4.csv'


TOWN_HOUSE_DIR = '../data/curated/seventh_clean/added_bus/town_house.csv'

TRAM_DIR = '../data/landing/public_transport/tram/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_METRO_TRAM_STOP.shp'


merge_nearby_tram(APAR_1_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/apartment_1.csv')
print('finish A1')
merge_nearby_tram(APAR_2_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/apartment_2.csv')
print('finish A2')
merge_nearby_tram(APAR_3_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/apartment_3.csv')
print('finish A3')
merge_nearby_tram(APAR_4_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/apartment_4.csv')
print('finish A4')
merge_nearby_tram(APAR_5_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/apartment_5.csv')
print('finish A5')
merge_nearby_tram(HOUSE_1_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/house_1.csv')
print('finish H1')
merge_nearby_tram(HOUSE_2_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/house_2.csv')
print('finish H2')
merge_nearby_tram(HOUSE_3_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/house_3.csv')
print('finish H3')
merge_nearby_tram(HOUSE_4_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/house_4.csv')
print('finish H4')
merge_nearby_tram(TOWN_HOUSE_DIR, TRAM_DIR, '../data/curated/seventh_clean/added_bus_tram/town_house.csv')
print('finish TH')

finish A1
finish A2
finish A3
finish A4
finish A5
finish H1
finish H2
finish H3
finish H4
finish TH


In [5]:
def merge_nearby_metro_train(input_file, transport_file, output_file):
#merge the nearest metro train station to the property dataset 
#input_file: path for the property file
#transport_file: path for the public transport file
#output_file: path for the merged file
    transport_data = gpd.read_file(transport_file)
    property_list = pd.read_csv(input_file)
    
    nearby_transport_coord=[]
    for count in range(len(property_list["Latitude"])):
        property_lat = float(property_list["Latitude"][count])
        property_long = float (property_list["longitude"][count])
        nearby_transport_name_coordinates=[]
        for count in range(len(transport_data['LONGITUDE'])):
            latitude_float = float(transport_data["LATITUDE"][count])
            longitude_float = float(transport_data["LONGITUDE"][count])
            name = transport_data["STOP_NAME"][count]
            distance = haversine(property_lat, property_long, latitude_float, longitude_float)
            if distance <= 10:
                nearby_transport_name_coordinates.append([name,latitude_float,longitude_float, distance])
        
        nearby_transport_coord.append(nearby_transport_name_coordinates)

    nearest=[]
    for i, nearby_transport_coord in enumerate(nearby_transport_coord):
        nearest_location = find_nearest_location(nearby_transport_coord)
        nearest.append(nearest_location)
        
    new_column_values = nearest  # Replace with your actual values
    property_list['Nearest_Metro_Train_Station'] = new_column_values

    if not os.path.exists("../data/curated/seventh_clean/added_bus_tram_metro"):
        os.makedirs("../data/curated/seventh_clean/added_bus_tram_metro")
    
    property_list.to_csv(output_file, index=False)

In [14]:
APAR_1_DIR = '../data/curated/seventh_clean/added_bus_tram/apartment_1.csv'
APAR_2_DIR = '../data/curated/seventh_clean/added_bus_tram/apartment_2.csv'
APAR_3_DIR = '../data/curated/seventh_clean/added_bus_tram/apartment_3.csv'
APAR_4_DIR = '../data/curated/seventh_clean/added_bus_tram/apartment_4.csv'
APAR_5_DIR = '../data/curated/seventh_clean/added_bus_tram/apartment_5.csv'

HOUSE_1_DIR = '../data/curated/seventh_clean/added_bus_tram/house_1.csv'
HOUSE_2_DIR = '../data/curated/seventh_clean/added_bus_tram/house_2.csv'
HOUSE_3_DIR = '../data/curated/seventh_clean/added_bus_tram/house_3.csv'
HOUSE_4_DIR = '../data/curated/seventh_clean/added_bus_tram/house_4.csv'

TOWN_HOUSE_DIR = '../data/curated/seventh_clean/added_bus_tram/town_house.csv'

METRO_TRAIN_DIR = '../data/landing/public_transport/metro_train/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_METRO_TRAIN_STATION.shp'

merge_nearby_metro_train(APAR_1_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/apartment_1.csv')
merge_nearby_metro_train(APAR_2_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/apartment_2.csv')
merge_nearby_metro_train(APAR_3_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/apartment_3.csv')
merge_nearby_metro_train(APAR_4_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/apartment_4.csv')
merge_nearby_metro_train(APAR_5_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/apartment_5.csv')
merge_nearby_metro_train(HOUSE_1_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/house_1.csv')
merge_nearby_metro_train(HOUSE_2_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/house_2.csv')
merge_nearby_metro_train(HOUSE_3_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/house_3.csv')
merge_nearby_metro_train(HOUSE_4_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/house_4.csv')
merge_nearby_metro_train(TOWN_HOUSE_DIR, METRO_TRAIN_DIR, '../data/curated/seventh_clean/added_bus_tram_metro/town_house.csv')

In [15]:
def merge_nearby_regional_train(input_file, transport_file, output_file):
#merge the nearest regional train station to the property dataset 
#input_file: path for the property file
#transport_file: path for the public transport file
#output_file: path for the merged file
    transport_data = gpd.read_file(transport_file)
    property_list = pd.read_csv(input_file)
    
    nearby_transport_coord=[]

    for count in range(len(property_list["Latitude"])):
        property_lat = float(property_list["Latitude"][count])
        property_long = float (property_list["longitude"][count])
        nearby_transport_name_coordinates=[]
        for count in range(len(transport_data['LONGITUDE'])):
            latitude_float = float(transport_data["LATITUDE"][count])
            longitude_float = float(transport_data["LONGITUDE"][count])
            name = transport_data["STOP_NAME"][count]
            distance = haversine(property_lat, property_long, latitude_float, longitude_float)
            if distance <= 30:
                nearby_transport_name_coordinates.append([name,latitude_float,longitude_float, distance])
        
        nearby_transport_coord.append(nearby_transport_name_coordinates)

    nearest=[]
    for i, nearby_transport_coord in enumerate(nearby_transport_coord):
        nearest_location = find_nearest_location(nearby_transport_coord)
        nearest.append(nearest_location)
        
    new_column_values = nearest  # Replace with your actual values
    property_list['Nearest_Regional_Train_Station'] = new_column_values


    if not os.path.exists("../data/curated/seventh_clean/final"):
        os.makedirs("../data/curated/seventh_clean/final")
    
    property_list.to_csv(output_file, index=False)

In [16]:
REGIONAL_TRAIN_RADIUS = 10

APAR_1_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/apartment_1.csv'
APAR_2_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/apartment_2.csv'
APAR_3_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/apartment_3.csv'
APAR_4_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/apartment_4.csv'
APAR_5_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/apartment_5.csv'

HOUSE_1_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/house_1.csv'
HOUSE_2_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/house_2.csv'
HOUSE_3_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/house_3.csv'
HOUSE_4_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/house_4.csv'

TOWN_HOUSE_DIR = '../data/curated/seventh_clean/added_bus_tram_metro/town_house.csv'

REGIONAL_TRAIN_DIR = '../data/landing/public_transport/regional_train/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_REGIONAL_TRAIN_STATION.shp'

merge_nearby_regional_train(APAR_1_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/apartment_1.csv')
merge_nearby_regional_train(APAR_2_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/apartment_2.csv')
merge_nearby_regional_train(APAR_3_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/apartment_3.csv')
merge_nearby_regional_train(APAR_4_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/apartment_4.csv')
merge_nearby_regional_train(APAR_5_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/apartment_5.csv')
merge_nearby_regional_train(HOUSE_1_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/house_1.csv')
merge_nearby_regional_train(HOUSE_2_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/house_2.csv')
merge_nearby_regional_train(HOUSE_3_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/house_3.csv')
merge_nearby_regional_train(HOUSE_4_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/house_4.csv')
merge_nearby_regional_train(TOWN_HOUSE_DIR, REGIONAL_TRAIN_DIR, '../data/curated/seventh_clean/final/town_house.csv')

In [20]:
df = pd.read_csv('../data/curated/seventh_clean/final/apartment_1.csv')

In [21]:
df

,name,Suburb,State,post,cost_text,Latitude,longitude,Beds,Baths,Parking,...,Population_density,School_with_rank,Total_employee_income_median,Total_employee_income_mean,Total_income_median,Total_income_mean,Nearest_Bus_Stop,Nearest_Tram_Stop,Nearest_Metro_Train_Station,Nearest_Regional_Train_Station
0,3605/439 Collins Street,Melbourne,VIC,3000,"4,500",-37.817688,144.959726,4,3,2,...,17196.298828,"[['Docklands Primary School', 1.7, None], ['Un...",NaN,NaN,NaN,NaN,"['Collins St/Queen St (Melbourne City)', -37.8...","['3-William St/Collins St (Melbourne City)', -...",['Flinders Street Railway Station (Melbourne C...,['Southern Cross Railway Station (Melbourne Ci...
1,5401/308 Exhibition Street,Melbourne,VIC,3000,"3,750",-37.808517,144.969628,3,2,4,...,13309.226562,"[['Carlton Gardens Primary School', 0.7, 138],...",NaN,NaN,NaN,NaN,['Little Lonsdale St/Exhibition St (Melbourne ...,['8-Exhibition St/La Trobe St (Melbourne City)...,['Parliament Railway Station (Melbourne City)'...,['Flinders Street Railway Station (Melbourne C...
2,4904/63 Latrobe St,Melbourne,VIC,3000,"2,900",-37.808203,144.969400,3,3,2,...,13309.226562,"[['Carlton Gardens Primary School', 0.7, 138],...",NaN,NaN,NaN,NaN,['Little Lonsdale St/Exhibition St (Melbourne ...,['8-Exhibition St/La Trobe St (Melbourne City)...,['Parliament Railway Station (Melbourne City)'...,['Flinders Street Railway Station (Melbourne C...
3,6703/500 Elizabeth Street,Melbourne,VIC,3000,"2,800",-37.807262,144.960271,3,3,1,...,33456.195312,"[['Carlton Gardens Primary School', 1.0, 138],...",NaN,NaN,NaN,NaN,"[""A'Beckett St/Queen St (Melbourne City)"", -37...",['7-Queen Victoria Market/Elizabeth St (Melbou...,['Melbourne Central Railway Station (Melbourne...,['Flinders Street Railway Station (Melbourne C...
4,4802/316-318 Queen Street,Melbourne,VIC,3000,2500,-37.811267,144.959261,3,3,2,...,33456.195312,"[['Carlton Gardens Primary School', 1.4, 138],...",NaN,NaN,NaN,NaN,"['La Trobe St/Queen St (Melbourne City)', -37....","['4-Queen St/La Trobe St (Melbourne City)', -3...","['Flagstaff Railway Station (Melbourne City)',...",['Southern Cross Railway Station (Melbourne Ci...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,818/118 Franklin Street,Melbourne,VIC,3000,55,-37.808205,144.958903,1,1,1,...,33456.195312,"[['Carlton Gardens Primary School', 1.2, 138],...",NaN,NaN,NaN,NaN,"[""A'Beckett St/Queen St (Melbourne City)"", -37...",['7-Queen Victoria Market/Elizabeth St (Melbou...,['Melbourne Central Railway Station (Melbourne...,['Southern Cross Railway Station (Melbourne Ci...
802,215/181 Exhibition Street,Melbourne,VIC,3000,55,-37.812011,144.970001,1,1,1,...,13309.226562,"[['Carlton Gardens Primary School', 1.1, 138],...",NaN,NaN,NaN,NaN,"['Exhibition St/Lonsdale St (Melbourne City)',...","['7-Russell St/Bourke St (Melbourne City)', -3...",['Parliament Railway Station (Melbourne City)'...,['Flinders Street Railway Station (Melbourne C...
803,CP 4010/120 Abeckett Street,Melbourne,VIC,3000,220,-37.809309,144.959739,1,1,1,...,33456.195312,"[['Carlton Gardens Primary School', 1.2, 138],...",NaN,NaN,NaN,NaN,"[""A'Beckett St/Queen St (Melbourne City)"", -37...",['5-Elizabeth St/La Trobe St (Melbourne City)'...,['Melbourne Central Railway Station (Melbourne...,['Flinders Street Railway Station (Melbourne C...
804,CP2709/31 ABeckett Street,Melbourne,VIC,3000,50,-37.808953,144.962623,1,1,1,...,33456.195312,"[['Carlton Gardens Primary School', 1.0, 138],...",NaN,NaN,NaN,NaN,['Old Melbourne Gaol/Russell St (Melbourne Cit...,"['6-Swanston St/La Trobe St (Melbourne City)',...",['Melbourne Central Railway Station (Melbourne...,['Flinders Street Railway Station (Melbourne C...
